In [1]:
import pyomo.environ as pyo
from pyomo.environ import *
m= pyo.ConcreteModel()

In [2]:
from Parameters import * 
db_connection = mysql.connector.connect(
host="localhost",
user="root",
passwd="morbius", # change password
auth_plugin='mysql_native_password'
)
db_cursor = db_connection.cursor(buffered=True)
db_cursor.execute("USE earthquake")

In [3]:
K = Parameters.num_of_immediate(db_cursor) #number of victims with condition 3 (immediate)
L = Parameters.num_of_delayed(db_cursor) #number of victims condition 2 (delay)
H = Parameters.num_of_hospitals(db_cursor) #operational hospitals
victim_dict = Parameters.victim_dict(db_cursor) #all victims
immediate_v_id_list, delayed_v_id_list = Parameters.victim_id_lists(victim_dict) #immediate and delayed victim id's
hospital_dict = Parameters.hospital_dict(db_cursor) 
hospital_id_list = Parameters.hospital_id_lists(hospital_dict) 
neighbourhood_list, hospital_list, distances = Parameters.distances(db_cursor, victim_dict, hospital_dict)
y_immediate_list, y_delayed_list = Parameters.y_lists(victim_dict) #age
t_immediate_list, t_delayed_list = Parameters.t_lists(victim_dict) #time elapsed
neighbourhood_immediate_list, neighbourhood_delayed_list = Parameters.neighbourhood_lists(victim_dict) 

##### Variables

In [4]:
#x: Binary variable. victim i to hospital j
#THE ONLY WAY I COULD FIND TO HAVE 2 INDEXED VARIABLES - DONT QUESTION IT TOO MUCH
m.I = pyo.RangeSet(0, len(immediate_v_id_list)-1)
m.D = pyo.RangeSet(0, len(delayed_v_id_list)-1)
m.J = pyo.RangeSet(0, len(hospital_id_list)-1)

m.K = np.ndarray(shape=(len(immediate_v_id_list),len(hospital_id_list)))
m.L = np.ndarray(shape=(len(delayed_v_id_list),len(hospital_id_list)))

def IJ_rule(m):
    return [(i,j) for i in range(len(m.I)) for j in range(len(m.K[i]))]

def DJ_rule(m):
    return [(d,j) for d in range(len(m.D)) for j in range(len(m.L[d]))]

m.immediate = Set(within=m.I*m.J, initialize=IJ_rule)
m.immediate_v = Var(m.immediate, within = Binary) #for immediate
m.delayed = Set(within=m.D*m.J, initialize=DJ_rule)
m.delayed_v = Var(m.delayed, within = Binary) #for delayed

##### Parameters

In [5]:
c_1 = 5 #constant 1
c_2 = 6 #constant 2
E = 100 #epsilon

##### Objective Function 


In [6]:
def obj_rule(m):
    immediate = 0
    for k in range(K):
        for j in range(H):
            neighbourhood_index = neighbourhood_list.index(neighbourhood_immediate_list[k])
            immediate += c_1*t_immediate_list[k]*y_immediate_list[k]*distances[neighbourhood_index][j]*m.immediate_v[k,j]
            
    delayed = 0
    for l in range(L):
        for j in range(H):
            neighbourhood_index = neighbourhood_list.index(neighbourhood_delayed_list[k])
            delayed += c_2*t_delayed_list[k]*y_delayed_list[k]*distances[neighbourhood_index][j]*m.delayed_v[l,j]
            
    penalty_immediate = K - sum(m.immediate_v[k,j] for k in range(K) for j in range(H))
    
    penalty_delayed = L - sum(m.delayed_v[l,j] for l in range(L) for j in range(H))
    
    return immediate + delayed + E*(c_1*penalty_immediate + c_2*penalty_delayed)

m.objective = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

##### Constraints

In [44]:
def constraint1(m, b): 
    return sum(m.x[(k, b)] for k in K) + sum(m.x[(l, b)] for l in L) <= B

m.Constraint1 = pyo.Constraint(j, rule = constraint1)

RuntimeError: Cannot add component 'Constraint1_index' (type <class 'pyomo.core.base.set.OrderedScalarSet'>) to block 'unknown': a component by that name (type <class 'pyomo.core.base.set.OrderedScalarSet'>) is already defined.

In [45]:
def constraint2(m, a): 
    return sum(m.x[(a, b)] for b in j ) <= 1 
m.Constraint2 = pyo.Constraint(i, rule = constraint2)

RuntimeError: Cannot add component 'Constraint2_index' (type <class 'pyomo.core.base.set.FiniteScalarSet'>) to block 'unknown': a component by that name (type <class 'pyomo.core.base.set.FiniteScalarSet'>) is already defined.

In [76]:
if K <= B:
    z = 1
else:
    z = 0

In [46]:
def constraint3(m): 
    return sum(m.x[(k, b)] for k in K for b in j ) >= z*p*immediate_victims + (1-z)*p*B
m.Constraint3 = pyo.Constraint(rule = constraint2)

    'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
ERROR: Rule failed when generating expression for Constraint Constraint3 with
    index None: KeyError: "Index '(None, 1)' is not valid for indexed
    component 'x'"
ERROR: Constructing component 'Constraint3' from data=None failed: KeyError:
    "Index '(None, 1)' is not valid for indexed component 'x'"


KeyError: "Index '(None, 1)' is not valid for indexed component 'x'"

In [ ]:
solver = SolverFactory('cplex_direct')
solution = solver.solve(m)
m.display()